In [104]:
# Імпорт необхідних бібліотек
from sklearn.metrics import silhouette_score, adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Завантажуємо та готуємо дані
data = pd.read_csv("Online-eCommerce.csv")
data = data.dropna()

# Перетворюємо колонку Order_Date у формат datetime
data['Order_Date'] = pd.to_datetime(data['Order_Date'], errors='coerce')
data = data.dropna(subset=['Order_Date'])

# Групуємо дані за клієнтами та обчислюємо ключові показники
customer_data = data.groupby('Customer_Name').agg({
    'Total_Sales': 'sum',
    'Quantity': 'sum',
    'Cost': 'mean',
    'Order_Date': lambda x: (pd.Timestamp.now() - x.max()).days
}).reset_index()

# Перейменовуємо колонки для зручності
customer_data.columns = [
    'Customer_Name', 'Total_Spent', 'Total_Quantity', 'Avg_Cost', 'Days_Since_Last_Purchase'
]

# Вибираємо числові ознаки
X = customer_data[['Total_Spent', 'Total_Quantity', 'Avg_Cost', 'Days_Since_Last_Purchase']]

# Масштабуємо дані
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Виконуємо кластеризацію K-means
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans_labels = kmeans.fit_predict(X_scaled)

# Виконуємо агломеративну кластеризацію
agg = AgglomerativeClustering(n_clusters=2)
agg_labels = agg.fit_predict(X_scaled)

# Виконуємо кластеризацію DBSCAN
dbscan = DBSCAN(eps=1.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(X_scaled)

# Обчислюємо метрики стійкості
silhouette_kmeans = silhouette_score(X_scaled, kmeans_labels)
ari_kmeans = adjusted_rand_score(kmeans_labels, kmeans_labels)  # Порівняння з собою
nmi_kmeans = normalized_mutual_info_score(kmeans_labels, kmeans_labels)

# Обчислюємо метрики для Agglomerative Clustering
silhouette_agg = silhouette_score(X_scaled, agg_labels)
ari_agg = adjusted_rand_score(agg_labels, agg_labels)  # Порівняння з собою
nmi_agg = normalized_mutual_info_score(agg_labels, agg_labels)

# Обчислюємо метрики для DBSCAN
silhouette_dbscan = silhouette_score(X_scaled, dbscan_labels)
ari_dbscan = adjusted_rand_score(dbscan_labels, dbscan_labels)  # Порівняння з собою
nmi_dbscan = normalized_mutual_info_score(dbscan_labels, dbscan_labels)

# Повертаємо результати для всіх алгоритмів
(silhouette_kmeans, ari_kmeans, nmi_kmeans,
 silhouette_agg, ari_agg, nmi_agg,
 silhouette_dbscan, ari_dbscan, nmi_dbscan)

C:\Users\911\anaconda3\envs\myenv\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


(0.40853986069765225,
 1.0,
 1.0,
 0.4009908863728576,
 1.0,
 1.0,
 0.2651327291027883,
 1.0,
 1.0)